In [1]:
import scipy.io
import entropy
import numpy as np
from scipy.ndimage import zoom
from skimage.transform import resize
import cv2
import os

In [2]:
mat=scipy.io.loadmat('Scratch\phi.mat')
data=mat['data']
actual_phi=mat['phi']
actual_theta=mat['theta']
actual_R=mat['R']
actual_traj=mat['Traj']
actual_t=mat['t']
entropy_traj=mat['actual_entropy_traj']
actual_L=mat['L']
actual_IE=mat['video_IE']
vr = [3840,2048]
theta,phi,R=entropy.xyz2thetaphi(data[:,1],data[:,2],data[:,3])
traj,x,y=entropy.evaluate_fix_map_traj(theta,phi,vr,0.025)
actual_entropy_traj,t=entropy.lzentropy_AE(traj)
vid_IE=np.zeros([len(traj),1])
L=[1,0]
for i in range(1,len(traj)):
    vid_IE[i],L=entropy.lzentropy_IE_rec(traj[0:i+1],L)

In [3]:
data[:,0]

array([ 0. ,  0.2,  0.4,  0.6,  0.8,  1. ,  1.2,  1.4,  1.6,  1.8,  2. ,
        2.2,  2.4,  2.6,  2.8,  3. ,  3.2,  3.4,  3.6,  3.8,  4. ,  4.2,
        4.4,  4.6,  4.8,  5. ,  5.2,  5.4,  5.6,  5.8,  6. ,  6.2,  6.4,
        6.6,  6.8,  7. ,  7.2,  7.4,  7.6,  7.8,  8. ,  8.2,  8.4,  8.6,
        8.8,  9. ,  9.2,  9.4,  9.6,  9.8, 10. , 10.2, 10.4, 10.6, 10.8,
       11. , 11.2, 11.4, 11.6, 11.8, 12. , 12.2, 12.4, 12.6, 12.8, 13. ,
       13.2, 13.4, 13.6, 13.8, 14. , 14.2, 14.4, 14.6, 14.8, 15. , 15.2,
       15.4, 15.6, 15.8, 16. , 16.2, 16.4, 16.6, 16.8, 17. , 17.2, 17.4,
       17.6, 17.8, 18. , 18.2, 18.4, 18.6, 18.8, 19. , 19.2, 19.4, 19.6,
       19.8, 20. , 20.2, 20.4, 20.6, 20.8, 21. , 21.2, 21.4, 21.6, 21.8,
       22. ])

In [4]:
AE,IE,L2=entropy.get_entropies(data[:,1],data[:,2],data[:,3],vr,0.025)

In [5]:
def get_video_dims(video_folder):
    video_files=os.listdir(video_folder)
    dims={}
    for video in video_files:
        vid_num=video.split("_")[1].split(".")[0]

        vid_path=os.path.join(video_folder,video)
        cap=cv2.VideoCapture(vid_path)
        vid_height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        vid_width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        dims[vid_num]=[int(vid_width),int(vid_height)]
    return dims

In [6]:
dataset_folder="D:\Jin_22"
traj_folder=os.path.join(dataset_folder,'video_data')
video_folder=os.path.join(dataset_folder,"Videos")
def get_entropies(src_folder,video_folder):
    src_files=os.listdir(src_folder)
    dims=get_video_dims(video_folder)
    for video in src_files:
        video_num=video.split('_')[1]
        vector_path=os.path.join(src_folder,video,f"{video}_unit_vectors.npy")
        unit_vectors=np.load(vector_path)
        AEs=[]
        IEs=[]
        print(f"Processing {video}")
        for user_path in unit_vectors:
            x,y,z=user_path[:,0],user_path[:,1],user_path[:,2]
            AE,IE,L2=entropy.get_entropies(x,z,y,dims[video_num])
            AEs.append(AE)
            IEs.append(IE)
        AEs=np.array(AEs)
        IEs=np.array(IEs)
        print(IEs.shape)
        return
        np.save(os.path.join(src_folder,video,f"Video_{video_num}_AEs.npy"),AEs)
        np.save(os.path.join(src_folder,video,f"Video_{video_num}_IEs.npy"),IEs)


In [7]:
get_entropies(traj_folder,video_folder)

Processing video_1
(99, 301, 1)


In [69]:
def get_video_dims(video_folder):
    dims={}
    videos=os.listdir(video_folder)
    for video in videos:
        vid_num=video.split(".")[0]

        vid_path=os.path.join(video_folder,video)
        cap=cv2.VideoCapture(vid_path)
        vid_height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        vid_width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        dims[vid_num]=[int(vid_width),int(vid_height)]
    return dims
    
dataset_folder='D:/CVPR18'
src_folder=os.path.join(dataset_folder,'sampled_dataset')
video_folder=os.path.join(dataset_folder,'Videos')
dst_folder=os.path.join(dataset_folder,'Trajectory_info')
def get_entropies(src_folder,video_folder):
    dims=get_video_dims(video_folder)
    #print(dims)
    src_files=os.listdir(src_folder)
    for video in src_files:
        if int(video)<126:
        #if int(video)==125:
            continue
        vector_path=os.path.join(src_folder,video,f"{video}_unit_vectors.npy")
        unit_vectors=np.load(vector_path)
        AEs=[]
        IEs=[]
        print(f"Processing {video}")
        for user_path in unit_vectors:
            x,y,z=user_path[:,0],user_path[:,1],user_path[:,2]
            AE,IE,L2=entropy.get_entropies(x,z,y,dims[video])
            AEs.append(AE)
            IEs.append(IE)
        AEs=np.array(AEs)
        IEs=np.array(IEs)
        np.save(os.path.join(src_folder,video,f"{video}_AEs.npy"),AEs)
        np.save(os.path.join(src_folder,video,f"{video}_IEs.npy"),IEs)


In [71]:
get_entropies(dst_folder,video_folder)

Processing 126
Processing 127
Processing 128
Processing 129
Processing 130
Processing 131
Processing 132
Processing 133
Processing 134
Processing 135
Processing 136
Processing 137
Processing 138
Processing 139
Processing 140
Processing 141
Processing 142
Processing 143
Processing 144
Processing 145
Processing 146
Processing 147
Processing 148
Processing 149
Processing 150
Processing 151
Processing 152
Processing 153
Processing 154
Processing 155
Processing 156
Processing 157
Processing 158
Processing 159
Processing 160
Processing 161
Processing 162
Processing 163
Processing 164
Processing 165
Processing 166
Processing 167
Processing 168
Processing 169
Processing 170
Processing 171
Processing 172
Processing 173
Processing 174
Processing 175
Processing 176
Processing 177
Processing 178
Processing 179
Processing 180
Processing 181
Processing 182
Processing 183
Processing 184
Processing 185
Processing 186
Processing 187
Processing 188
Processing 189
Processing 190
Processing 191
Processing

### Fan NOSSDAV

In [21]:
def get_video_dims(video_folder):
    dims={}
    videos=os.listdir(video_folder)
    for video in videos:
        vid_name=video.split(".")[0]

        vid_path=os.path.join(video_folder,video)
        cap=cv2.VideoCapture(vid_path)
        vid_height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        vid_width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        dims[vid_name]=[int(vid_width),int(vid_height)]
    return dims
    
dataset_folder='D:/Fan_NOSSDAV_17'
src_folder=os.path.join(dataset_folder,'sampled_dataset')
video_folder=os.path.join(dataset_folder,'Videos')
dst_folder=os.path.join(dataset_folder,'Trajectory_info')
def get_entropies(src_folder,video_folder):
    dims=get_video_dims(video_folder)
    #print(dims)
    src_files=os.listdir(src_folder)
    
    for video in src_files:
        vector_path=os.path.join(src_folder,video,f"{video}_unit_vectors.npy")
        unit_vectors=np.load(vector_path)
        AEs=[]
        IEs=[]
        print(f"Processing {video}")
        for user_path in unit_vectors:
            x,y,z=user_path[:,0],user_path[:,1],user_path[:,2]
            AE,IE,L2=entropy.get_entropies(x,z,y,dims[video])
            AEs.append(AE)
            IEs.append(IE)
        AEs=np.array(AEs)
        IEs=np.array(IEs)
        np.save(os.path.join(src_folder,video,f"{video}_AEs.npy"),AEs)
        np.save(os.path.join(src_folder,video,f"{video}_IEs.npy"),IEs)


In [22]:
get_entropies(dst_folder,video_folder)

Processing coaster
Processing coaster2
Processing diving
Processing drive
Processing game
Processing landscape
Processing pacman
Processing panel
Processing ride
Processing sport


### PAMI18

In [5]:
def get_video_dims(video_folder):
    dims={}
    videos=os.listdir(video_folder)
    for video in videos:
        vid_name=video.split(".")[0]

        vid_path=os.path.join(video_folder,video)
        cap=cv2.VideoCapture(vid_path)
        vid_height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        vid_width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        dims[vid_name]=[int(vid_width),int(vid_height)]
    return dims
    
dataset_folder='D:/PAMI18'
src_folder=os.path.join(dataset_folder,'sampled_dataset')
video_folder=os.path.join(dataset_folder,'Videos')
dst_folder=os.path.join(dataset_folder,'video_data')
def get_entropies(src_folder,video_folder):
    dims=get_video_dims(video_folder)
    #print(dims)
    src_files=os.listdir(src_folder)
    
    for video in src_files:
        vector_path=os.path.join(src_folder,video,f"{video}_unit_vectors.npy")
        unit_vectors=np.load(vector_path)
        AEs=[]
        IEs=[]
        print(f"Processing {video}")
        for user_path in unit_vectors:
            x,y,z=user_path[:,0],user_path[:,1],user_path[:,2]
            AE,IE,L2=entropy.get_entropies(x,z,y,dims[video])
            AEs.append(AE)
            IEs.append(IE)
        AEs=np.array(AEs)
        IEs=np.array(IEs)
        np.save(os.path.join(src_folder,video,f"{video}_AEs.npy"),AEs)
        np.save(os.path.join(src_folder,video,f"{video}_IEs.npy"),IEs)


In [7]:
dims=get_video_dims(video_folder)

In [16]:
dims

{'A380': [3840, 1920],
 'AcerEngine': [3840, 1920],
 'AcerPredator': [3840, 1920],
 'AirShow': [3840, 1920],
 'BFG': [3840, 2160],
 'Bicycle': [3840, 1920],
 'BlueWorld': [3840, 1920],
 'BTSRun': [3840, 1920],
 'Camping': [3840, 1920],
 'CandyCarnival': [3840, 1920],
 'Castle': [3840, 1920],
 'Catwalks': [3840, 1920],
 'CMLauncher': [3840, 1920],
 'CMLauncher2': [3840, 1920],
 'CS': [3840, 1920],
 'DanceInTurn': [3840, 1920],
 'Dancing': [3840, 1920],
 'DrivingInAlps': [3840, 1920],
 'Egypt': [3840, 1920],
 'F5Fighter': [2880, 1440],
 'Flight': [2880, 1440],
 'FULLdata_per_video_frame': [0, 0],
 'GalaxyOnFire': [3840, 2160],
 'Graffiti': [3840, 1920],
 'GTA': [3840, 1920],
 'Guitar': [3840, 1920],
 'HondaF1': [3840, 1920],
 'InsideCar': [3840, 1920],
 'IRobot': [3840, 2160],
 'KasabianLive': [3840, 1920],
 'KingKong': [3840, 1080],
 "Let'sNotBeAloneTonight": [3840, 1920],
 'Lion': [3840, 2160],
 'LoopUniverse': [4096, 2048],
 'Manhattan': [3840, 1920],
 'MC': [3840, 1920],
 'MercedesBe

In [9]:
get_entropies(dst_folder,video_folder)

Processing A380
Processing AcerEngine
Processing AcerPredator
Processing AirShow
Processing BFG
Processing Bicycle
Processing BlueWorld
Processing BTSRun
Processing Camping
Processing CandyCarnival
Processing Castle
Processing Catwalks
Processing CMLauncher
Processing CMLauncher2
Processing CS
Processing DanceInTurn
Processing Dancing
Processing DrivingInAlps
Processing Egypt
Processing F5Fighter
Processing Flight
Processing GalaxyOnFire
Processing Graffiti
Processing GTA
Processing Guitar
Processing HondaF1
Processing InsideCar
Processing IRobot
Processing KasabianLive
Processing KingKong
Processing Lion
Processing LoopUniverse
Processing Manhattan
Processing MC
Processing MercedesBenz
Processing Motorbike
Processing Murder
Processing NotBeAloneTonight
Processing Orion
Processing Parachuting
Processing Parasailing
Processing Pearl
Processing Predator
Processing ProjectSoul
Processing Rally
Processing RingMan
Processing RioOlympics
Processing Roma
Processing Shark
Processing Skiing
Pro

### MMSys18

In [17]:
def get_video_dims(video_folder):
    dims={}
    videos=os.listdir(video_folder)
    for video in videos:
        vid_name=video.split(".")[0]

        vid_path=os.path.join(video_folder,video)
        cap=cv2.VideoCapture(vid_path)
        vid_height=cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
        vid_width=cap.get(cv2.CAP_PROP_FRAME_WIDTH)
        dims[vid_name]=[int(vid_width),int(vid_height)]
    return dims
    
dataset_folder='D:/MMSys18'
src_folder=os.path.join(dataset_folder,'sampled_dataset')
video_folder=os.path.join(dataset_folder,'Videos')
dst_folder=os.path.join(dataset_folder,'video_data')
def get_entropies(src_folder,video_folder):
    dims=get_video_dims(video_folder)
    print(dims)
    src_files=os.listdir(src_folder)
    
    for video in src_files:
        vector_path=os.path.join(src_folder,video,f"{video}_unit_vectors.npy")
        unit_vectors=np.load(vector_path)
        AEs=[]
        IEs=[]
        print(f"Processing {video}")
        for user_path in unit_vectors:
            x,y,z=user_path[:,0],user_path[:,1],user_path[:,2]
            AE,IE,L2=entropy.get_entropies(x,z,y,dims[video],scale=0.1)
            AEs.append(AE)
            IEs.append(IE)
        AEs=np.array(AEs)
        IEs=np.array(IEs)
        np.save(os.path.join(src_folder,video,f"{video}_AEs.npy"),AEs)
        np.save(os.path.join(src_folder,video,f"{video}_IEs.npy"),IEs)


In [18]:
get_entropies(dst_folder,video_folder)

{'1_PortoRiverside': [640, 340], '2_Diner': [640, 340], '3_PlanEnergyBioLab': [640, 340], '4_Ocean': [640, 340], '5_Waterpark': [640, 340], '6_DroneFlight': [640, 340], '7_GazaFishermen': [640, 340], '8_Sofa': [640, 340], '9_MattSwift': [640, 340], '10_Cows': [640, 340], '11_Abbottsford': [640, 340], '12_TeatroRegioTorino': [640, 340], '13_Fountain': [640, 340], '14_Warship': [640, 340], '15_Cockpit': [640, 340], '16_Turtle': [640, 340], '17_UnderwaterPark': [640, 340], '18_Bar': [640, 340], '19_Touvet': [640, 340]}
Processing 10_Cows
Processing 11_Abbottsford
Processing 12_TeatroRegioTorino
Processing 13_Fountain
Processing 14_Warship
Processing 15_Cockpit
Processing 16_Turtle
Processing 17_UnderwaterPark
Processing 18_Bar
Processing 19_Touvet
Processing 1_PortoRiverside
Processing 2_Diner
Processing 3_PlanEnergyBioLab
Processing 4_Ocean
Processing 5_Waterpark
Processing 6_DroneFlight
Processing 7_GazaFishermen
Processing 8_Sofa
Processing 9_MattSwift
